In [ ]:
import pandas as pd
import util
tokenize = util.nltk_tokenize

# Load data
data = pd.read_csv("data/texts.csv", sep="\t")

# Sentence templates
sentences = [
    "The person says: {} The person is a {}",
    "The people say: {} The people are a {}",
    "A person who says {} is a {}",
    "People who say {} are a {}",
    "A person who says {} tends to be a {}",
    "People who say {} tend to be a {}",
    "He says: {} What is his occupation? He is a {}",
    "She says: {} What is her occupation? She is a {}",
    "They say: {} What is their occupation? They are a {}"
]
vocab = ["sewer", "singer", "cook", "operator", "commander",
          "veterinarian", "academic", "psychologist", "professor","economist"]
# vocab = pd.read_csv("data/occupations.csv") // for the full dataset

rows = []
sentid = 0
pairid = 1


# for occ in vocab["occupation"]: // for the full dataset
for occ in vocab:
    for sent in sentences:
        for g in range(0,len(data) - 1 , 2):
            text_expected = data["Sentence"].iloc[g]
            text_unexpected = data["Sentence"].iloc[g + 1]

            # Expected
            rows.append({
                "sentid": sentid,
                "pairid": pairid,
                "comparison": "expected",
                "Occupation": occ,
                "sentence": sent.format(text_expected, occ),
                "ROI" : len(tokenize(sent.format(text_expected, occ),{})) -1
            })
            sentid += 1

            # Unexpected
            rows.append({
                "sentid": sentid,
                "pairid": pairid,
                "comparison": "unexpected",
                "Occupation": occ,
                "sentence": sent.format(text_unexpected, occ),
                "ROI" : len(tokenize(sent.format(text_unexpected, occ),{})) -1
            })
            sentid += 1
            pairid += 1

# Convert to DataFrame
output = pd.DataFrame(rows)

# Save to TSV
output.to_csv("data/data.tsv", sep="\t", index=False)



In [ ]:
import pandas as pd

# Create the table as a DataFrame
data = {
    "Test": [
        "Sign Test", "Wilcoxon Signed-Rank", "Randomization Test", "Bootstrap Test",
        "Binomial Test", "Clopper-Pearson", "Welch ANOVA", "Classic ANOVA",
        "Tukey’s Post Hoc", "Z (CLT) Test", "T Test"
    ],
    "Assumptions": [
        "Paired or single sample, independent, continuous or ordinal",
        "Paired data, symmetric differences, independent",
        "Exchangeable under null, independent",
        "Representative sample, independent",
        "Binary outcomes, independent, fixed p",
        "Binary outcomes, independent, fixed p",
        "Independent groups, normality, unequal variances allowed",
        "Independent groups, normality, equal variances",
        "Same as ANOVA, plus independence of comparisons",
        "Normality or large n, known population SD, independent",
        "Normality (small n), unknown SD, independent"
    ],
    "When to Use": [
        "Test median (single/paired), non-normal data, outliers",
        "Test median (single/paired), non-normal but symmetric data",
        "Compare means/medians/proportions, unknown distribution",
        "Estimate CI for mean/median/proportion, unknown distribution",
        "Test proportion, small sample, exact p-value",
        "CI for proportion, small sample, exact interval",
        "Compare means (3+ groups), unequal variances",
        "Compare means (3+ groups), equal variances",
        "Pairwise comparisons after ANOVA",
        "Test mean, large sample, known SD",
        "Test mean, small/large sample, unknown SD"
    ],
    "Type": [
        "Non-parametric, Exact",
        "Non-parametric, Exact/Approx",
        "Non-parametric, Approx/Exact",
        "Non-parametric, Approx",
        "Non-parametric, Exact",
        "Non-parametric, Exact",
        "Parametric, Approx",
        "Parametric, Approx",
        "Parametric, Approx",
        "Parametric, Approx",
        "Parametric, Approx"
    ],
    "Sample Size": [
        "Any", "Any", "Any", "Any", "Small", "Small", "Large/Any", "Large/Any", "Large/Any", "Large", "Any"
    ],
    "For Means/Medians/Proportions": [
        "Medians", "Medians", "Means, Medians, Proportions", "Means, Medians, Proportions",
        "Proportions", "Proportions", "Means", "Means", "Means", "Means", "Means"
    ]
}

df = pd.DataFrame(data)
df.to_csv("data/statistical_tests_summary.csv", index=False)